In [29]:
import numpy as np
from nba_api.stats.endpoints import leaguegamelog
import time
def get_all_games(start_year=1946, end_year=2023):
  results = []
  for year in range(start_year, end_year):
    season = '{}-{}'.format(year, str(year + 1)[-2:])
    res = leaguegamelog.LeagueGameLog(counter=10000, season=season)
    res_frame = res.get_data_frames()[0]
    results.append(res_frame)
    time.sleep(1)
  return results
    
seasons = get_all_games(2022, 2024)

In [30]:
# games retrieved include the stats for both teams as separate rows, so put them in 'home' or 'away' respectively
def map_home_and_away(seasons):
  results_all_season = []
  for season in seasons:
    results_for_season = []
    results_for_season.append(season[season.duplicated(subset='GAME_ID', keep='first')])
    results_for_season.append(season[season.duplicated(subset='GAME_ID', keep='last')])

    results_all_season.append(results_for_season)
    
  return results_all_season

# seasons_home_and_away = map_home_and_away(results)

In [31]:
# TODO: string still present in fields SEASON_ID, GAME_ID, GAME_DATE, WL
# SEASON_ID = drop
# GAME_ID = drop
# GAME_DATE = figure this out
# WL = [0, 1]

# map seasonid to 0..len(season)
def preprocess(seasons):
  
  # drop unnecessary columns
  for season in seasons:
    season.drop(columns=['TEAM_ABBREVIATION', 'TEAM_NAME',\
                         'MATCHUP', 'VIDEO_AVAILABLE'], inplace=True)

  # convert WL to 0 and 1
  WL = {'L': 0., 'W': 1.}
  for season in seasons:
    season['WL'] = season['WL'].map(WL.get)

  # add home and away dimension
  seasons = map_home_and_away(seasons)

  # map season_id to index
  season_index = []
  for season in seasons:
  
  
  # do whatever else with pandas before they are turned into numpy
  # ...
  
  # seasons_np = np.array((len(seasons), 2, 10000, 24), dtype=np.float32)
  for i, (home, away) in enumerate(seasons):
    game_id_mapping.append(home['GAME_ID'])
    season_id_mapping.append(home['SEASON_ID'])
    team_id_mapping.append(home['TEAM_ID'])
    home.drop(columns=['GAME_ID', 'SEASON_ID', 'TEAM_ID'], inplace=True)
    away.drop(columns=['GAME_ID', 'SEASON_ID', 'TEAM_ID'], inplace=True)
    home.to_numpy(dtype=np.float32)
    away.to_numpy(dtype=np.float32)
    
  seasons_np = np.array(seasons, dtype=np.float32)
  
  return seasons_np

seasons_np = preprocess(seasons)

         TEAM_ID     GAME_ID   GAME_DATE WL  MIN  FGM  FGA  FG_PCT  FG3M  \
0     1610612747  0022200002  2022-10-18  L  240   40   94   0.426    10   
1     1610612744  0022200002  2022-10-18  W  240   45   99   0.455    16   
2     1610612755  0022200001  2022-10-18  L  240   40   80   0.500    13   
3     1610612738  0022200001  2022-10-18  W  240   46   82   0.561    12   
4     1610612753  0022200003  2022-10-19  L  240   42   86   0.488    11   
...          ...         ...         ... ..  ...  ...  ...     ...   ...   
2455  1610612751  0022201217  2023-04-09  L  240   35   83   0.422    12   
2456  1610612762  0022201228  2023-04-09  L  240   51  101   0.505    10   
2457  1610612737  0022201216  2023-04-09  L  240   44   97   0.454    11   
2458  1610612738  0022201216  2023-04-09  W  240   42   89   0.472    25   
2459  1610612766  0022201218  2023-04-09  W  240   37   81   0.457     6   

      FG3A  ...  OREB  DREB  REB  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  
0       40 

/tmp/ipykernel_48290/909118183.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home.drop(columns='GAME_ID', inplace=True)
/tmp/ipykernel_48290/909118183.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away.drop(columns='GAME_ID', inplace=True)
/tmp/ipykernel_48290/909118183.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home.drop(columns='GAME_ID', inplace=True)
/tmp/ipykernel_48290/909118183.py:25: SettingWithCopyWarni

ValueError: could not convert string to float: '2022-10-18'

In [7]:
WL = {'L': 0., 'W': 1.}
# arr = ['L', 'W', 'B']
# .map(WL::get)
r = ['L', 'W']
r.max(WL:get)
# WL::get

SyntaxError: invalid syntax (2746070439.py, line 5)

In [60]:
seasons[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2460 entries, 0 to 2459
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SEASON_ID          2460 non-null   object 
 1   TEAM_ID            2460 non-null   int64  
 2   TEAM_ABBREVIATION  2460 non-null   object 
 3   TEAM_NAME          2460 non-null   object 
 4   GAME_ID            2460 non-null   object 
 5   GAME_DATE          2460 non-null   object 
 6   MATCHUP            2460 non-null   object 
 7   WL                 2460 non-null   object 
 8   MIN                2460 non-null   int64  
 9   FGM                2460 non-null   int64  
 10  FGA                2460 non-null   int64  
 11  FG_PCT             2460 non-null   float64
 12  FG3M               2460 non-null   int64  
 13  FG3A               2460 non-null   int64  
 14  FG3_PCT            2460 non-null   float64
 15  FTM                2460 non-null   int64  
 16  FTA                2460 